In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
filename = 'genius_table_values_and_lyrics'
infile = open(filename, 'rb')
df = pickle.load(infile)
infile.close()

In [3]:
df.head()

,url,side_table,lyrics
0,https://genius.com/Kanye-west-gone-lyrics,{'Written By': ['https://genius.com/artists/Ch...,"[[Produced by Kanye West], [Intro], Wished I h..."
1,https://genius.com/Nicki-minaj-i-am-your-leade...,{'Written By': ['https://genius.com/artists/Fe...,"[[Verse 1: Nicki Minaj], Uh, Look sucker, this..."
2,https://genius.com/Lil-kim-crush-on-you-remix-...,{'Written By': ['https://genius.com/artists/An...,"[[Intro: Lil' Cease & Lil' Kim], Undeas!, Lil ..."
3,https://genius.com/Camron-hey-ma-lyrics,{'Written By': ['https://genius.com/artists/D-...,"[[Chorus: Freekey Zekey (Toya)], Hey ma (wassu..."
4,https://genius.com/Camron-killa-cam-lyrics,{'Written By': ['https://genius.com/artists/Th...,"[[Chorus throughout Intro], Killa Cam, Killa C..."


In [4]:
# turn list of lists into string for countVectorizer
def lol(list_of_lists):
    return " ".join([ item for sublist in list_of_lists for item in sublist ])

In [5]:
values =[list(x.values()) for x in df['side_table']]

In [6]:
a = [ lol(i) for i in values ]

In [7]:
df["table_as_string"]=pd.Series(a)

In [8]:
test = df.dropna()

In [9]:
test.shape

(187694, 4)

In [10]:
test = test.reset_index()

In [11]:
test.head()

,index,url,side_table,lyrics,table_as_string
0,0,https://genius.com/Kanye-west-gone-lyrics,{'Written By': ['https://genius.com/artists/Ch...,"[[Produced by Kanye West], [Intro], Wished I h...",https://genius.com/artists/Chuck-willis https:...
1,1,https://genius.com/Nicki-minaj-i-am-your-leade...,{'Written By': ['https://genius.com/artists/Fe...,"[[Verse 1: Nicki Minaj], Uh, Look sucker, this...",https://genius.com/artists/Fernando-garibay ht...
2,2,https://genius.com/Lil-kim-crush-on-you-remix-...,{'Written By': ['https://genius.com/artists/An...,"[[Intro: Lil' Cease & Lil' Kim], Undeas!, Lil ...",https://genius.com/artists/Andreao-fanatic-hea...
3,3,https://genius.com/Camron-hey-ma-lyrics,{'Written By': ['https://genius.com/artists/D-...,"[[Chorus: Freekey Zekey (Toya)], Hey ma (wassu...",https://genius.com/artists/D-r-period https://...
4,4,https://genius.com/Camron-killa-cam-lyrics,{'Written By': ['https://genius.com/artists/Th...,"[[Chorus throughout Intro], Killa Cam, Killa C...",https://genius.com/artists/The-heatmakerz http...


In [12]:
test = test.drop(columns=["index"],axis=1)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer =CountVectorizer()

In [14]:
corpus = test['table_as_string'].values

In [15]:
X = vectorizer.fit_transform(corpus)

In [16]:
X

<187694x69860 sparse matrix of type '<class 'numpy.int64'>'
	with 1538732 stored elements in Compressed Sparse Row format>

In [17]:
X[0]

<1x69860 sparse matrix of type '<class 'numpy.int64'>'
	with 71 stored elements in Compressed Sparse Row format>

In [49]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300, n_iter=50, random_state=42)
X_svd = svd.fit_transform(X)  

## Trying the Annoy Library
#### annoy test

In [22]:
!pip install --user annoy

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
import annoy

In [24]:
## from annoy import AnnoyIndex
# import random

# f = 40
# t = AnnoyIndex(f)  # Length of item vector that will be indexed
# for i in range(1000):
#     v = [random.gauss(0, 1) for z in range(f)]
#     t.add_item(i, v)

# t.build(10) # 10 trees
# t.save('test.ann')

# # ...

# u = AnnoyIndex(f)
# u.load('test.ann') # super fast, will just mmap the file
# print(u.get_nns_by_item(0, 10)) # will find the 1000 nearest neighbors

## Annoy on my data

In [25]:
songs = test["url"]
songs.shape

(187694,)

In [50]:
from annoy import AnnoyIndex
t = AnnoyIndex(300)

for i in range(0, len(songs)):
    t.add_item(i, list(X_svd[i]))
t.build(100)

True

In [38]:
test.index[test["url"] =="https://genius.com/Rakim-when-i-b-on-tha-mic-lyrics"]

Int64Index([2629], dtype='int64')

In [51]:
t.save('test.ann')
u = AnnoyIndex(300)
u.load('test.ann') # super fast, will just mmap the file
playlist =u.get_nns_by_item(2629, 10) # will find the 1000 nearest neighbors

In [52]:
playlist

[31, 81, 211, 227, 518, 564, 594, 870, 1172, 1286]

In [53]:
for i in playlist:
#     print(i)
   print(songs[i],"\n")

https://genius.com/Camron-losing-weight-pt-2-lyrics 

https://genius.com/Jay-z-kill-jay-z-lyrics 

https://genius.com/Clipse-when-the-last-time-lyrics 

https://genius.com/Clipse-hot-damn-lyrics 

https://genius.com/Jadakiss-we-gonna-make-it-lyrics 

https://genius.com/Juvenile-juvenile-on-fire-lyrics 

https://genius.com/The-game-the-city-lyrics 

https://genius.com/Cormega-rap-basquiat-lyrics 

https://genius.com/Madvillain-rhinestone-cowboy-lyrics 

https://genius.com/Jamie-foxx-storm-forecast-lyrics 



In [54]:
test.index[test["url"] =="https://genius.com/Kanye-west-stronger-lyrics"]

Int64Index([2291], dtype='int64')

In [56]:
playlist2 =u.get_nns_by_item(2291, 10) # will find the 1000 nearest neighbors
for i in playlist2:
#     print(i)
   print(songs[i],"\n")

https://genius.com/Kanye-west-stronger-lyrics 

https://genius.com/Three-6-mafia-doe-boy-fresh-lyrics 

https://genius.com/Mike-stud-amanda-bynes-versace-remix-lyrics 

https://genius.com/Lil-jon-and-the-east-side-boyz-get-low-lyrics 

https://genius.com/Vado-my-bae-lyrics 

https://genius.com/Bloodhound-gang-chew-toy-lyrics 

https://genius.com/Outkast-gasoline-dreams-lyrics 

https://genius.com/D12-leave-dat-boy-alone-lyrics 

https://genius.com/Lil-kim-fuck-you-1996-lyrics 

https://genius.com/Count-bass-d-aural-sects-lyrics 



In [ ]:
test.index[test["url"] =="https://genius.com/Kanye-west-stronger-lyrics"]

## Hyper parameter search
500 components, 75 iterations 100 trees,  the same as 300 components 50 iters 100 trees, 300x15x35,


In [66]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=15, random_state=1)
X_svd = svd.fit_transform(X)  
t = AnnoyIndex(100)

for i in range(0, len(songs)):
    t.add_item(i, list(X_svd[i]))
t.build(200)
playlist3 =u.get_nns_by_item(2291, 10) # will find the 1000 nearest neighbors
for i in playlist3:
#     print(i)
   print(songs[i],"\n")

https://genius.com/Kanye-west-stronger-lyrics 

https://genius.com/Three-6-mafia-doe-boy-fresh-lyrics 

https://genius.com/Mike-stud-amanda-bynes-versace-remix-lyrics 

https://genius.com/Lil-jon-and-the-east-side-boyz-get-low-lyrics 

https://genius.com/Vado-my-bae-lyrics 

https://genius.com/Bloodhound-gang-chew-toy-lyrics 

https://genius.com/Outkast-gasoline-dreams-lyrics 

https://genius.com/D12-leave-dat-boy-alone-lyrics 

https://genius.com/Lil-kim-fuck-you-1996-lyrics 

https://genius.com/Count-bass-d-aural-sects-lyrics 

